In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
import _pickle as pkl
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<h2> Data Clean </h2>

### 定義清除html的tag以及將辭意重複的字詞作處理

In [2]:
def preprocessor(text):
    # remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text

def tokenizer(text):
    return re.split('\s+', text.strip())

def tokenizer_stem(text):
    porter = PorterStemmer()
    return [porter.stem(word) for word in re.split('\s+', text.strip())]

def tokenizer_stem_nostop(text):
    stop = stopwords.words('english')
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

def get_stream(path, size):
    for chunk in pd.read_csv(path, chunksize=size):
        yield chunk

<h3> Feature engineering</h3> 

挑選出我們需要的feature，包括:
    1. author: 作者姓名
    2. channel: 文章類別
    3. title: 文章標題
    4. topic: 文章細項分類
    5. weekday: 文章在星期幾發布

In [3]:
def fetch_datetime(text=[]):

    day = []
    
    for tx in text:
        soup = BeautifulSoup(tx, "lxml")
        if soup.time.has_attr('datetime'):
            date = soup.time.attrs['datetime']
            day.append(' '+ date[0:3])
        else:
            day.append(' fuckday')
    return day

def fetch_channel(text=[]):
    
    channels = []
    
    for tx in text:
        soup = BeautifulSoup(tx, "lxml")
        channel = soup.article['data-channel']
        channels.append(channel)
    return channels

def fetch_img_count(text=[]):

    count = []
    
    for tx in text:
        
        soup = BeautifulSoup(tx, "lxml")
        c = 0
        find_all_images = soup.find_all('img')
        
        for i in find_all_images:
            c = c+1
        count.append(c)
    return count

def fetch_topics(text=[]):

    topics = []
    
    for tx in text:
        soup = BeautifulSoup(tx, "lxml")
        footer = soup.footer
        ta = footer.find_all('a')
        topic = []

        for t in ta:
            topic.append(t.get_text())
        ff = ' '.join(topic)
        topics.append(ff)

    return topics

def fetch_authors(text=[]):

    authors = []
    
    for tx in text:
        soup = BeautifulSoup(tx, "lxml")
        footer = soup.span
        if footer != None:
            ta = footer.findAll('a')
            author = []
            for t in ta:
                author.append(t.get_text())
            if len(author) == 0:
                ff = 'NaN'
            else:
                ff = ''.join(author)
        else:
            ff = 'NaN'
        authors.append(ff)

    return authors

def fetch_titles(text=[]):
    
    titles = []
    
    for tx in text:
        soup = BeautifulSoup(tx, "lxml")
        footer = soup.h1
        if footer != None:
            ff = footer.get_text()
        else:
            ff = 'NaN'
        titles.append(ff)
        
    return titles
def fetch_social_media_count(text=[]):
    count = []
    for tx in text:
        soup = BeautifulSoup(tx, "lxml")
        c = 0
        for frame in soup("iframe"):
#             print(frame.get('src').split("."))
            if frame.get('src').find("youtube") != None:
                c = c+1
            elif frame.get('src').find("instagram") != None:
                c = c+1
            elif frame.get('src').find("vine") != None:
                c= c+1
            # apply new media here

        count.append(c)
    return count

In [5]:
# read training data
df = pd.read_csv('./input/train.csv')
# display(df.head(5))
print(fetch_social_media_count(df[0:10]['Page content']))
print(df[0:10]['Popularity'])

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [ ]:
# fatch deature
topic_batch   = fetch_topics            (df[:]['Page content'])
channel_batch = fetch_channel           (df[:]['Page content'])
weekday_batch = fetch_datetime          (df[:]['Page content'])
author_batch  = fetch_authors           (df[:]['Page content'])
img_batch     = fetch_img_count         (df[:]['Page content'])
title_batch   = fetch_titles            (df[:]['Page content'])
media_batch   = fetch_social_media_count(df[:]['Page content'])

In [13]:
# re-consturct training data
df_feature = pd.DataFrame({'Id':df.Id[:],
                           'Popularity':df.Popularity[:],
                           'topic':topic_batch,
                           'channel':channel_batch,
                           'weekday':weekday_batch,
                           'author':author_batch,
                           'img count':img_batch,
                           'title':title_batch,
                           'media count': media_batch})

In [14]:
display(df_feature.head(5))

,Id,Popularity,author,channel,img count,media count,title,topic,weekday
0,0,-1,NaN,world,1,0,NASA's Grand Challenge: Stop Asteroids From De...,Asteroid Asteroids challenge Earth Space U.S. ...,Wed
1,1,1,Christina Warren,tech,2,0,Google's New Open Source Patent Pledge: We Won...,Apps and Software Google open source opn pledg...,Thu
2,2,1,Sam Laird,entertainment,2,25,Ballin': 2014 NFL Draft Picks Get to Choose Th...,Entertainment NFL NFL Draft Sports Television,Wed
3,3,-1,Sam Laird,watercooler,1,21,Cameraperson Fails Deliver Slapstick Laughs,Sports Video Videos Watercooler,Fri
4,4,-1,Connor Finnegan,entertainment,52,1,NFL Star Helps Young Fan Prove Friendship With...,Entertainment instagram instagram video NFL Sp...,Thu


In [45]:
# sc = StandardScaler()
# print(np.array(df_feature[:]['media count'].values.reshape(-1, 1)))
# sc.fit(np.array(df_feature[:]['media count'].values.reshape(-1, 1)))
# media_feature = sc.transform(df_feature['media count'])

### 對挑選出來的feature根據性質做onehot encoding 或是 featurre hashing

In [46]:
# feature pre-processing

# topic to 4000-dim vector
hashvec = HashingVectorizer(n_features=4000,
                            preprocessor=preprocessor,
                            tokenizer=tokenizer_stem_nostop)

topic_hash = hashvec.transform(df_feature['topic'])
print(topic_hash.shape)

# ohe channel
onehotencoderchannel = OneHotEncoder(handle_unknown='ignore')
data_str_ohe_channel = onehotencoderchannel.fit_transform(df_feature['channel'].values.reshape(-1,1)).toarray()
print(data_str_ohe_channel.shape)

# ohe weekday
onehotencoderweekday = OneHotEncoder(handle_unknown='ignore')
data_str_ohe_weekday = onehotencoderweekday.fit_transform(df_feature['weekday'].values.reshape(-1,1)).toarray()
print(data_str_ohe_weekday.shape)

# ohe author
onehotencoderauthor = OneHotEncoder(handle_unknown='ignore')
data_str_ohe_author = onehotencoderauthor.fit_transform(df_feature['author'].values.reshape(-1,1)).toarray()
print(data_str_ohe_author.shape)

title_hash = hashvec.transform(df_feature['title'])


print(title_hash.shape)

(27643, 4000)
(27643, 33)
(27643, 7)
(27643, 428)
(27643, 4000)


In [47]:
df_X_train = np.concatenate([topic_hash.toarray(), data_str_ohe_channel,
                             data_str_ohe_weekday, data_str_ohe_author, title_hash.toarray()], axis=1)

df_y_train = df_feature['Popularity'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(df_X_train, df_y_train, test_size=0.1)

### 使用SGDClassifier 對處理好的資料training

In [48]:
SGDLR = SGDClassifier(loss='log', max_iter=1000, alpha=0.0002)

total_epoch = 50
for epoch in range(total_epoch):
    SGDLR.partial_fit(X_train, y_train, classes=np.array([-1, 1]))
    print('epoch ', epoch, roc_auc_score(y_train, SGDLR.predict_proba(X_train)[:,1]))
print("training finished")

SGDLR_pred_decision = SGDLR.predict(X_test)
acc = accuracy_score(y_test, SGDLR_pred_decision)
print('Accuracy: %f' % acc)

epoch  0 0.6108992752641363
epoch  1 0.6558416782037642
epoch  2 0.6597213930803631
epoch  3 0.6680220271703503
epoch  4 0.6709409469509022
epoch  5 0.6792859240376927
epoch  6 0.6841521800972586
epoch  7 0.6770870996196645
epoch  8 0.6901256213182667
epoch  9 0.6877716545873693
epoch  10 0.6959724652225988
epoch  11 0.6918381013140646
epoch  12 0.6935495729268448
epoch  13 0.6964670124528386
epoch  14 0.6955382270536095
epoch  15 0.6862720920801592
epoch  16 0.6904274445762474
epoch  17 0.6966094013061314
epoch  18 0.6921931943441529
epoch  19 0.6947486179028024
epoch  20 0.6916715306603285
epoch  21 0.6891888205668456
epoch  22 0.6921299764857363
epoch  23 0.6925094840200412
epoch  24 0.6927043217182413
epoch  25 0.6929294690829153
epoch  26 0.6925108349947252
epoch  27 0.6978375730377153
epoch  28 0.6931457349202412
epoch  29 0.6925063877670573
epoch  30 0.6960193162489592
epoch  31 0.6924378435778293
epoch  32 0.6933631836684148
epoch  33 0.6954631865507097
epoch  34 0.692120875182

### 使用LogisticRegression對處理好的資料training

In [49]:
LR = LogisticRegression(max_iter=1000, C=0.3)
LR.fit(X_train, y_train)

LR_pred_probablity = LR.predict_proba(X_test)[:,1]
LR_pred_decision   = LR.predict(X_test)

roc = roc_auc_score(y_test, LR_pred_probablity)
print('ROC_AUC : %f' % roc)
acc = accuracy_score(y_test, LR_pred_decision)
print('Accuracy: %f' % acc)

ROC_AUC : 0.563354
Accuracy: 0.546474


In [50]:
# read testing data
df_test = pd.read_csv('./test.csv')

test_topic_batch   = fetch_topics   (df_test[:]['Page content'])
test_channel_batch = fetch_channel  (df_test[:]['Page content'])
test_weekday_batch = fetch_datetime (df_test[:]['Page content'])
test_author_batch  = fetch_authors  (df_test[:]['Page content'])
test_img_batch     = fetch_img_count(df_test[:]['Page content'])
test_title_batch   = fetch_titles   (df_test[:]['Page content'])

df_test_feature = pd.DataFrame({'Id'        : df_test.Id[:],
                                'topic'     : test_topic_batch,
                                'channel'   : test_channel_batch,
                                'weekday'   : test_weekday_batch,
                                'author'    : test_author_batch,
                                'img count' : test_img_batch,
                                'title'     : test_title_batch})

display(df_test_feature.head(5))

,Id,author,channel,img count,title,topic,weekday
0,27643,Sam Laird,entertainment,1,Soccer Star Gets Twitter Death Threats After T...,Entertainment Music One Direction soccer Sports,Mon
1,27644,Stan Schroeder,tech,3,Google Glass Gets an Accessory Store,Gadgets glass Google Google Glass Google Glass...,Thu
2,27645,Todd Wasserman,business,2,OUYA Gaming Console Already Sold Out on Amazon,amazon amazon kindle Business Gaming,Tue
3,27646,Neha Prakash,film,1,'Between Two Ferns' Mocks Oscar Nominees,Between Two Ferns Movies The Oscars Oscars 201...,Wed
4,27647,Josh Dickey,entertainment,1,'American Sniper' Trailer: Looks Like Eastwood...,American Sniper Awards Bradley Cooper clint ea...,Fri


In [51]:
topic_test_hash = hashvec.transform(df_test_feature['topic'])
print(topic_test_hash.shape)

test_data_str_ohe_channel = onehotencoderchannel.transform(df_test_feature['channel'].values.reshape(-1,1)).toarray()
print(test_data_str_ohe_channel.shape)

test_data_str_ohe_weekday = onehotencoderweekday.transform(df_test_feature['weekday'].values.reshape(-1,1)).toarray()  
print(test_data_str_ohe_weekday.shape)

test_data_str_ohe_author = onehotencoderauthor.transform(df_test_feature['author'].values.reshape(-1,1)).toarray()  
print(test_data_str_ohe_author.shape)

title_test_hash = hashvec.transform(df_test_feature['title'])
print(title_test_hash.shape)

X_test = np.concatenate([topic_test_hash.toarray(), test_data_str_ohe_channel,
                         test_data_str_ohe_weekday, test_data_str_ohe_author, title_test_hash.toarray()], axis=1)

(11847, 4000)
(11847, 33)
(11847, 7)
(11847, 428)
(11847, 4000)


###  對處理好的資料，使用SGDClassifier、LogisticRegression預測

In [52]:
df_out = pd.DataFrame({"Id":df_test.Id, "Popularity":SGDLR.predict_proba(X_test)[:,1]})
df_out.to_csv("./SGDLR.csv", index=None)
print("SGDLR Done.")

SGDLR Done.


In [53]:
df_out = pd.DataFrame({"Id":df_test.Id, "Popularity":LR.predict_proba(X_test)[:,1]})
df_out.to_csv("./LR.csv", index=None)
print("LR Done.")

LR Done.


In [55]:
s = (SGDLR.predict_proba(X_test)[:,1] + LR.predict_proba(X_test)[:,1]) / 2
df_out = pd.DataFrame({"Id":df_test.Id, "Popularity":s})
df_out.to_csv("./mix_LR_SGDLR.csv", index=None)
print("mix_LR_SGDLR Done.")

mix_LR_SGDLR Done.


### 後來決定多使用RandomForest來做預測，並將結果和logistic regession做結合

In [56]:
from sklearn.ensemble import RandomForestClassifier
RandomForest = RandomForestClassifier(n_estimators=200, max_depth=20, random_state=0)
# batch_size = 1000
# stream = get_stream(path='./train.csv', size=batch_size)
# classes = np.array([-1, 1])
# train_auc, val_auc = [], []
# we use one batch for training and another for validation in each iteration
# per_round_iters = int((27000+batch_size-1)/batch_size)

# total_epoch = 25

# X_train, y_train = cat, df_feature[:]['Popularity']

RandomForest.fit(X_train, y_train)
print('epoch ', epoch, roc_auc_score(y_train, RandomForest.predict_proba(X_train)[:,1]))

print("training finished")




# LR = LogisticRegression(max_iter=1000, C=0.3)
# LR.fit(X_train, y_train)

# LR_pred_probablity = LR.predict_proba(X_test)[:,1]
# LR_pred_decision   = LR.predict(X_test)

# roc = roc_auc_score(y_test, LR_pred_probablity)
# print('ROC_AUC : %f' % roc)
# acc = accuracy_score(y_test, LR_pred_decision)
# print('Accuracy: %f' % acc)

epoch  49 0.8032993820467267
training finished


In [57]:
s = (RandomForest.predict_proba(X_test)[:,1] + LR.predict_proba(X_test)[:,1]) / 2
df_out = pd.DataFrame({"Id":df_test.Id, "Popularity":s})
df_out.to_csv("./mix_LR_RandomForest.csv", index=None)
print("mix_LR_RandomForest Done.")

mix_LR_RandomForest Done.


In [58]:
df_out = pd.DataFrame({"Id":df_test.Id, "Popularity":RandomForest.predict_proba(X_test)[:,1]})
df_out.to_csv("./HUANGRandomForest.csv", index=None)
print("HUANGRandomForest Done.")

HUANGRandomForest Done.


## Report
### 組別: 育榮我大哥
### 成員: 108062648 黃彥儒、108062510 劉志容、108133505 朱信承、108062630 楊書瑋

<h2> Date Perprocessing </h2>
<h3> Clean </h3>  

    我們將整篇文章內容透過 beautiful soup 來 parse html tag，
    並將文字一併轉成小寫後進行 tonkenize。

<h3> Feature engineering</h3> 

    在資料前處理的方面，我們挑選了以下幾個feature 作為我們training、testing dataset 的 feature:
    
    1. author: 作者姓名
    2. channel: 文章類別
    3. title: 文章標題
    4. topic: 文章細項分類
    5. weekday: 文章在星期幾發布
    
    其中考慮到資料的特性，我們將author、channel、weekday 做了 one-hot encoding，並且將title以及topic分別做Feature hashing之後，將這些vector concate起來，產生training set、testing set。
    
<h2>How did you build the classifier, e.g. model, training algorithm, special techniques, etc?</h2>

    在挑選classifer 方面，我們使用了logistic regression跟randomforest，將這兩種classifer 得到的 predict_proba 取平均。這邊想特別提一下logistic regression，原本我們使用的是scikit learn 的 SGDClassifier，但是經過比較發現logistic regression 的表現較好，我們猜測可能是因為兩個得到的結果都是線性，加上SGDClassifier是用SGD，在兩種classifer都已經收斂的情形下，logistic regression表現會較好。
    
<h2>Conclusions</h2>

    我們這次在kaggle的競賽上，在public跟private的表現落差不小，探究原因，我們猜測可能是我們最後在極致追求在public的測資上取得些微的進步，造成classifer 跟 feature的挑選實際上都是去 fit public data，但實際上還是應該要使用例如cross_validation 之類的方法來挑選才恰當。